In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

CREATE FLAG PARAMATER

In [0]:
dbutils.widgets.text('incremental_flag','0')

In [0]:
incremental_flag= dbutils.widgets.get('incremental_flag')
print(incremental_flag)

#CREATING DIMENSIONS

fetch relative colum


In [0]:
df_source = spark.sql('''
select DiSTINCT (BRANCH_ID) as Branch_ID,BRANCH_NAME from parquet.`abfss://silver@nvcardatalake.dfs.core.windows.net/carsales`''')

dim branch sink - initial and incremental

In [0]:
if spark.catalog.tableExists('car_catalog.gold.dim_branch'):
    df_sink=spark.sql(
    ''' SELECT  dim_branch_key,BRANCH_ID,BRANCH_NAME
    FROM car_catalog.gold.dim_branch
     ''')
else:    df_sink=spark.sql(
    ''' SELECT 1 as dim_branch_key,BRANCH_ID,BRANCH_NAME
    FROM parquet.`abfss://silver@nvcardatalake.dfs.core.windows.net/carsales`
    WHERE 1=0 ''')


In [0]:
df_sink.display()

### Filtering new record vs old record

In [0]:
df_filter = df_source.join(df_sink,df_source["BRANCH_ID"]  == df_sink["BRANCH_ID"],'left').select(df_source["BRANCH_ID"],df_source["BRANCH_NAME"],df_sink.dim_branch_key)
df_filter.display()

### df_filter_old

In [0]:
df_filter_old = df_filter.filter(col('dim_branch_key').isNotNull())
df_filter_old.display()

### df_filter_new

In [0]:
df_filter_new= df_filter.filter(col('dim_branch_key').isNull())
df_filter_new.display()

### Create surrogate key

**fetch the max surrogate key from existing table**

In [0]:
if (incremental_flag == '0'):
   max_value=1
else:
    max_value_df= spark.sql("select max(dim_branch_key) from cars_catalog.gold.dim_branch")
    max_value= max_value_df.collect()[0][0]+1


In [0]:
df_filter_new = df_filter_new.withColumn('dim_branch_key',max_value+ monotonically_increasing_id())
df_filter_new.display()

### final df filter - df_old+df_new

In [0]:
df_final= df_filter_new.union(df_filter_old)

In [0]:
df_final.display()

# scd-1 upsert


In [0]:
from delta import DeltaTable

In [0]:
if spark.catalog.tableExists('cars_catalog.gold.dim_branch'):
    delta_tbl = DeltaTable.forPath(spark,"abfss://gold@nvcardatalake.dfs.core.windows.net/dim_branch")
    delta_tbl.alias("trg").merge(df_final.alias("src"),"trg.dim_branch_key= src.dim_branch_key")\
                            .whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
    
else:
    df_final.write.format("delta").mode('overwrite').option('path',"abfss://gold@nvcardatalake.dfs.core.windows.net/dim_branch").saveAsTable('cars_catalog.gold.dim_branch')